# Introduction
I use this notebook multiple times across the life of project, both as a starting overview and a point of reference between various cleaning and transformation steps.

It uses Google Colab form entries and a personalized version of the [cookiecutter data science](https://drivendata.github.io/cookiecutter-data-science/) file structure to simplify file selection.

In addition to basic built-in `pandas` overview tools, like `.info()` and `.describe()`, I added my own functions for common tasks including:
*  Creating a singular table of the same data from `.info()` and `.describe()`
    * Writing the table to `.csv`
    * Writing the table to `.xlsx` with frozen column headers and a `Notes` column, which I use to take notes about processing steps
*  Displaying an overview of unique values by column

The final step utilizes the `dataprep.eda` library to generate an HTML report with additional insights.

# Set Up

## Authorize Google Drive
Follow pop up prompts to authorize Drive access. May not work with non-Chrome browsers depending on ad block and privacy settings.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
cd '/content/drive/MyDrive/data_analysis'

/content/drive/MyDrive/data_analysis


## Library imports

In [3]:
#general analysis
import pandas as pd
import pprint as ppr
import re
import numpy as np

import os

#stop words counter
from collections import Counter

## Display Preferences

In [4]:
#current preferences
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None) #change column display width
#pd.set_option('display.precision', 2) #displays 2 decimal places on all numbers
pd.set_option('display.float_format',  '{:.2f}'.format)
pd.set_option('display.memory_usage', 'deep')

# File Read In
This section uses parametized forms in Google Colab. Before running this on an existing notebook, please **File > Save a copy in Drive** and work from that copy to avoid overwriting existing work.  You will likely need to rename your copy (**File > Rename**) and move it to the appropriate project folder (**File > Move**).

In [33]:
#@title ## File Selection

#@markdown ---
#@markdown ### Select or enter a file path:
project = "dunnhumby" #@param ["project_01", "project_02"] {allow-input: true}
folder = "data/raw" #@param ["data/raw", "data/interim", "data/processed", "data/meta"] {allow-input: true}
#@markdown ---

In [6]:
#print a list of file names for copy/pasting into file selection
directory_files = os.listdir(f"{project}/{folder}/")

for file in directory_files:
  print(file[:-4])
  print(file[-4:])
  print()

transactions_200819
.csv

transactions_200818
.csv

transactions_200817
.csv



In [7]:
#@markdown ---
#@markdown ### Enter a file name and extension
file_name = "transactions_200819" #@param {type:"string"}
file_ext = ".csv" #@param [".csv", ".txt"] {allow-input: true}

### Read into pandas dataframe

In [8]:
file_path = f"{project}/{folder}/{file_name}{file_ext}"

In [9]:
df = pd.read_csv(file_path,
#                        usecols= cols,
#                      sep='\t',
#                        nrows=100,
#                       engine='python',
#                     encoding='ISO-8859-1'
                        )

# DataFrame Overview

## Row and Column Count

In [10]:
df.shape

(270686, 22)

## `.info()`

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270686 entries, 0 to 270685
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SHOP_WEEK                 270686 non-null  int64  
 1   SHOP_DATE                 270686 non-null  int64  
 2   SHOP_WEEKDAY              270686 non-null  int64  
 3   SHOP_HOUR                 270686 non-null  int64  
 4   QUANTITY                  270686 non-null  int64  
 5   SPEND                     270686 non-null  float64
 6   PROD_CODE                 270686 non-null  object 
 7   PROD_CODE_10              270686 non-null  object 
 8   PROD_CODE_20              270686 non-null  object 
 9   PROD_CODE_30              270686 non-null  object 
 10  PROD_CODE_40              270686 non-null  object 
 11  CUST_CODE                 218707 non-null  object 
 12  CUST_PRICE_SENSITIVITY    218707 non-null  object 
 13  CUST_LIFESTAGE            192280 non-null  o

## Descriptive Stats

In [12]:
df.describe(include='all')

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
count,270686.00,270686.00,270686.00,270686.00,270686.00,270686.00,270686,270686,270686,270686,270686,218707,218707,192280,270686.00,270686,270686,270686,270686,270686,270686,270686
unique,NaN,NaN,NaN,NaN,NaN,NaN,4997,250,90,31,9,17707,4,6,NaN,3,4,4,5,761,4,12
top,NaN,NaN,NaN,NaN,NaN,NaN,PRD0903052,CL00063,DEP00019,G00007,D00002,CUST0000743378,MM,OT,NaN,L,MM,Top Up,Fresh,STORE02561,LS,N03
freq,NaN,NaN,NaN,NaN,NaN,NaN,5898,11995,22165,59853,131115,118,101186,59338,NaN,190616,142509,118574,134507,1098,169396,27042
mean,200819.00,20080693.04,4.02,14.92,1.51,1.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994111700415452.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.00,25.71,2.00,3.67,2.03,2.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,235900.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,200819.00,20080630.00,1.00,8.00,1.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994111700000005.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,200819.00,20080701.00,2.00,12.00,1.00,0.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994111700213501.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,200819.00,20080703.00,4.00,15.00,1.00,1.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994111700416417.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,200819.00,20080705.00,6.00,18.00,1.00,2.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994111700621883.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Testing

## Data Types, Memory Usage, Nulls, Value Counts

In [13]:
#pandas documention re memory usage: base-2 representation; i.e. 1KB = 1024 bytes

In [14]:
def get_dataframe_info(df):
    """
Recreates column-wise info from df.info() as a DataFrame to allow for easier viewing from CSV
    """
    df_dtypes = pd.DataFrame(df.dtypes, columns=['Data Types'])

    df_memory_usage = df.memory_usage(index=False, deep=True).to_frame(name='Memory (Bytes)')
    df_memory_usage['Memory (MB)'] = df_memory_usage['Memory (Bytes)']/1024/1024

    df_percent_null = ((1 - df.count() / len(df)) * 100).to_frame(name='Percent Null')

    df_described = df.describe(include='all').T


    df_info = pd.concat([df_dtypes, df_memory_usage, df_percent_null, df_described], axis=1)

    # Reassign column names
    new_column_names =  {'count': 'Count Null',
                        'unique': 'Unique Counts',
                        'top': 'Top Value',
                        'freq': 'Frequency',
                        'mean': 'Mean',
                        'std': 'Standard Deviation',
                        'min': 'Minimum',
                        '25%': '25%',
                        '50%': '50%',
                        '75%': '75%',
                        'max': 'Maximum'}
    df_info = df_info.rename(columns=new_column_names).rename_axis('Column Names')

#    df_info = df_info.set_index('column_names')

    return df_info

In [15]:
df_info = get_dataframe_info(df)

In [16]:
df_info.style.set_sticky(axis='index')

,Data Types,Memory (Bytes),Memory (MB),Percent Null,Count Null,Unique Counts,Top Value,Frequency,Mean,Standard Deviation,Minimum,25%,50%,75%,Maximum
Column Names,,,,,,,,,,,,,,,
SHOP_WEEK,int64,2165488,2.065170,0.000000,270686.000000,nan,nan,nan,200819.000000,0.000000,200819.000000,200819.000000,200819.000000,200819.000000,200819.000000
SHOP_DATE,int64,2165488,2.065170,0.000000,270686.000000,nan,nan,nan,20080693.042027,25.711046,20080630.000000,20080701.000000,20080703.000000,20080705.000000,20080706.000000
SHOP_WEEKDAY,int64,2165488,2.065170,0.000000,270686.000000,nan,nan,nan,4.017360,1.997138,1.000000,2.000000,4.000000,6.000000,7.000000
SHOP_HOUR,int64,2165488,2.065170,0.000000,270686.000000,nan,nan,nan,14.924924,3.674051,8.000000,12.000000,15.000000,18.000000,21.000000
QUANTITY,int64,2165488,2.065170,0.000000,270686.000000,nan,nan,nan,1.512106,2.030642,1.000000,1.000000,1.000000,1.000000,692.000000
SPEND,float64,2165488,2.065170,0.000000,270686.000000,nan,nan,nan,1.913799,2.938752,0.000000,0.770000,1.230000,2.090000,366.840000
PROD_CODE,object,18135962,17.295801,0.000000,270686,4997,PRD0903052,5898,nan,nan,nan,nan,nan,nan,nan
PROD_CODE_10,object,17323904,16.521362,0.000000,270686,250,CL00063,11995,nan,nan,nan,nan,nan,nan,nan
PROD_CODE_20,object,17594590,16.779509,0.000000,270686,90,DEP00019,22165,nan,nan,nan,nan,nan,nan,nan


In [17]:
#write to csv file
file_path = f"{project}/data/meta/{file_name}_overview"
df_info.to_csv(file_path + ".csv", index=False)

In [18]:
#add a `notes` column for use with in a spreadsheet
df_info.insert(loc=0,
               column='Notes',
               value = '')

In [19]:
#write to excel file
df_info.to_excel(file_path + ".xlsx",
                 sheet_name=f'Overview {file_name}',
                 freeze_panes=(1,2)
                 )

## Correlation Table
For numeric columns only

In [20]:
df.corr(numeric_only=True)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,BASKET_ID
SHOP_WEEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SHOP_DATE,NaN,1.00,0.41,-0.00,-0.00,-0.00,-0.01
SHOP_WEEKDAY,NaN,0.41,1.00,-0.01,-0.00,0.00,-0.00
SHOP_HOUR,NaN,-0.00,-0.01,1.00,-0.02,0.00,-0.07
QUANTITY,NaN,-0.00,-0.00,-0.02,1.00,0.22,-0.01
SPEND,NaN,-0.00,0.00,0.00,0.22,1.00,0.00
BASKET_ID,NaN,-0.01,-0.00,-0.07,-0.01,0.00,1.00


## Head and Tail Rows
First and last 10 rows

In [21]:
df.head(10)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200819,20080704,6,20,1,1.77,PRD0900008,CL00042,DEP00011,G00004,D00002,CUST0000017325,MM,YA,994111700179457,L,UM,Top Up,Fresh,STORE00001,LS,E02
1,200819,20080702,4,14,1,1.27,PRD0900013,CL00015,DEP00004,G00003,D00001,CUST0000644893,LA,PE,994111700586989,L,MM,Full Shop,Mixed,STORE00001,LS,E02
2,200819,20080702,4,21,1,0.92,PRD0900014,CL00089,DEP00025,G00008,D00002,CUST0000029792,LA,YA,994111700187610,S,LA,Small Shop,Fresh,STORE00001,LS,E02
3,200819,20080701,3,15,1,0.71,PRD0900043,CL00148,DEP00052,G00015,D00003,CUST0000525126,LA,YA,994111700508988,L,LA,Full Shop,Fresh,STORE00001,LS,E02
4,200819,20080702,4,14,3,3.12,PRD0900055,CL00230,DEP00081,G00027,D00008,CUST0000644893,LA,PE,994111700586989,L,MM,Full Shop,Mixed,STORE00001,LS,E02
5,200819,20080705,7,21,3,3.12,PRD0900055,CL00230,DEP00081,G00027,D00008,CUST0000309908,UM,OT,994111700369756,M,UM,Small Shop,Grocery,STORE00001,LS,E02
6,200819,20080705,7,14,1,2.14,PRD0900063,CL00185,DEP00062,G00018,D00004,CUST0000418887,LA,NaN,994111700440704,L,MM,Top Up,Mixed,STORE00001,LS,E02
7,200819,20080704,6,17,1,1.33,PRD0900064,CL00209,DEP00069,G00022,D00005,CUST0000017325,MM,YA,994111700179454,M,LA,Top Up,Fresh,STORE00001,LS,E02
8,200819,20080701,3,13,1,1.05,PRD0900077,CL00150,DEP00052,G00015,D00003,CUST0000710863,LA,YF,994111700630067,L,MM,Full Shop,Mixed,STORE00001,LS,E02
9,200819,20080702,4,17,1,0.31,PRD0900083,CL00113,DEP00040,G00011,D00003,CUST0000206560,UM,YF,994111700303093,L,UM,Full Shop,Mixed,STORE00001,LS,E02


In [22]:
df.tail(10)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
270676,200819,20080703,5,15,1,1.55,PRD0904038,CL00067,DEP00019,G00007,D00002,NaN,NaN,NaN,994111700127061,L,UM,Full Shop,Mixed,STORE01062,SS,E02
270677,200819,20080702,4,19,1,2.06,PRD0904797,CL00160,DEP00054,G00016,D00003,CUST0000440778,MM,OF,994111700454856,L,MM,Full Shop,Fresh,STORE01062,SS,E02
270678,200819,20080704,6,21,3,7.53,PRD0903360,CL00100,DEP00033,G00009,D00002,CUST0000964754,MM,YF,994111700795841,M,UM,Small Shop,Fresh,STORE01062,SS,E02
270679,200819,20080705,7,18,1,0.41,PRD0902832,CL00222,DEP00076,G00023,D00005,CUST0000151037,LA,YF,994111700266853,M,MM,Top Up,Mixed,STORE01062,SS,E02
270680,200819,20080706,1,14,1,0.39,PRD0900780,CL00043,DEP00011,G00004,D00002,NaN,NaN,NaN,994111700127192,L,MM,Full Shop,Mixed,STORE01062,SS,E02
270681,200819,20080703,5,15,3,0.39,PRD0902428,CL00222,DEP00076,G00023,D00005,NaN,NaN,NaN,994111700127061,L,UM,Full Shop,Mixed,STORE01062,SS,E02
270682,200819,20080701,3,14,3,5.07,PRD0902348,CL00092,DEP00027,G00008,D00002,CUST0000870442,LA,YF,994111700733938,L,LA,Top Up,Mixed,STORE01062,SS,E02
270683,200819,20080701,3,14,1,0.38,PRD0904799,CL00092,DEP00027,G00008,D00002,CUST0000870442,LA,YF,994111700733938,L,LA,Top Up,Mixed,STORE01062,SS,E02
270684,200819,20080705,7,13,1,0.41,PRD0901943,CL00043,DEP00011,G00004,D00002,NaN,NaN,NaN,994111700167348,M,UM,Top Up,Fresh,STORE01062,SS,E02
270685,200819,20080705,7,13,1,0.93,PRD0903966,CL00068,DEP00019,G00007,D00002,NaN,NaN,NaN,994111700167348,M,UM,Top Up,Fresh,STORE01062,SS,E02


## Duplicated Rows

In [23]:
df[df.duplicated(keep=False)]

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION


## Null Columns
List of columns with all NaN values

In [24]:
list(df.columns[df.isnull().all(axis=0)])

[]

# Columns

## List of Column Names

In [25]:
list(df.columns)

['SHOP_WEEK',
 'SHOP_DATE',
 'SHOP_WEEKDAY',
 'SHOP_HOUR',
 'QUANTITY',
 'SPEND',
 'PROD_CODE',
 'PROD_CODE_10',
 'PROD_CODE_20',
 'PROD_CODE_30',
 'PROD_CODE_40',
 'CUST_CODE',
 'CUST_PRICE_SENSITIVITY',
 'CUST_LIFESTAGE',
 'BASKET_ID',
 'BASKET_SIZE',
 'BASKET_PRICE_SENSITIVITY',
 'BASKET_TYPE',
 'BASKET_DOMINANT_MISSION',
 'STORE_CODE',
 'STORE_FORMAT',
 'STORE_REGION']

## Column Overview
This function loops over each column to produce the following info:
* name
* count of unique values
* datatype
* string to copy into a new cell, for displaying all value counts for a column

For value counts under the cutoff:
* a transposed display frame of all unique values

For value counts over the cutoff:
* transposed display frames of the top and bottom 10 values and counts


In [26]:
def column_overview(df, columns, cutoff=100):
    """
    Display column name, count of unique values, and an easy to read dataframe of individual unique values and their counts

    Parameters
    ----------
    df: dataframe
    columns: list
    cutoff: int, maximum unique value count to display
    Recommend string or object columns with unclear or low expected unique values.

    Returns
    -------
    Prints out one result for each column in the provided list.
    """
    for col in columns:
        print("\n")
        print("Column name: " + col)
        num_unique = str(df[col].nunique())
        print(f"Number of Unique Values: {num_unique}")
        col_datatype = str(df[col].dtype)
        print(f"Column Datatype: {col_datatype}")
        print("\n")

        if int(num_unique) < cutoff:
          print("use line below for vertical results")
          print(f"pd.DataFrame(df['{col}'].value_counts(dropna=False))")
          display(pd.DataFrame(df[col].value_counts(dropna=False)).T)
          print(u'\u2500' * 80)
        else:
          print(f"more than {cutoff} results, showing Top 10 and Bottom 10")
          print("use line below for complete results")
          print(f"pd.DataFrame(df['{col}'].value_counts(dropna=False))")
          print("\n")
          print(f"Top 10 Unique Values of {col}")
          display(pd.DataFrame(df[col].value_counts(dropna=False).head(10)).T)
          print(f"Bottom 10 Unique Values of {col}")
          display(pd.DataFrame(df[col].value_counts(dropna=False).tail(10)).T)
          print(u'\u2500' * 80)


In [27]:
column_overview(df, df.columns, 200)



Column name: SHOP_WEEK
Number of Unique Values: 1
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_WEEK'].value_counts(dropna=False))


,200819
SHOP_WEEK,270686


────────────────────────────────────────────────────────────────────────────────


Column name: SHOP_DATE
Number of Unique Values: 7
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_DATE'].value_counts(dropna=False))


,20080705,20080701,20080703,20080630,20080702,20080704,20080706
SHOP_DATE,39402,39232,39034,38478,38444,38387,37709


────────────────────────────────────────────────────────────────────────────────


Column name: SHOP_WEEKDAY
Number of Unique Values: 7
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_WEEKDAY'].value_counts(dropna=False))


...

────────────────────────────────────────────────────────────────────────────────


Column name: STORE_FORMAT
Number of Unique Values: 4
Column Datatype: object


use line below for vertical results
pd.DataFrame(df['STORE_FORMAT'].value_counts(dropna=False))


,LS,MS,XLS,SS
STORE_FORMAT,169396,58353,23167,19770


────────────────────────────────────────────────────────────────────────────────


Column name: STORE_REGION
Number of Unique Values: 12
Column Datatype: object


use line below for vertical results
pd.DataFrame(df['STORE_REGION'].value_counts(dropna=False))


,N03,N01,S02,W02,N02,S01,S03,W01,E01,E03,W03,E02
STORE_REGION,27042,26754,26619,26108,22566,22130,21855,21585,19880,19554,18335,18258


────────────────────────────────────────────────────────────────────────────────


# Data Prep Library
Installs [dataprep library](https://dataprep.ai/) and runs dataprep.eda to produce HTML report with additional dataset information, including plots, word counts, and 'column insights'.

May take 10+ minutes to complete, or timeout if dataset is too large or messy.

In [28]:
!pip install dataprep

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 66.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 576.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.24-cp310-cp310-linux_x86_64.whl size=1252693 sha256=2bc2e0cf1d93

In [29]:
from dataprep.eda import create_report

report = create_report(df, title=f"{file_name} EDA Report")

/usr/local/lib/python3.10/dist-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/usr/local/lib/python3.10/dist-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/usr/local/lib/python3.10/dist-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({col: [nrows - npresent]}, index=["Others"]))
/usr/local/lib/python3.10/dist-packages/dataprep/eda/distribution/render.py:274: FutureWarning: The frame.append method is depreca

In [30]:
os.chdir(f'/content/drive/MyDrive/data_analysis/{project}/notebooks/eda')

In [31]:
report.save()

Report has been saved to /content/drive/MyDrive/data_analysis/dunnhumby/notebooks/eda/report.html!


In [32]:
old_report_file_name = r'report.html'
new_report_file_name = f'{folder[5:len(folder)]}_{file_name}_EDA.html'
os.rename(old_report_file_name, new_report_file_name)